In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import sigmoid_kernel

In [2]:
root_dj = "/Users/alper/Documents/DJ"
root_analysis = os.path.join(root_dj,'Analysis')

df = pd.read_csv(os.path.join(root_analysis,'Tracks.csv'))
df.head()

,Unnamed: 0,TITLE,ARTIST,ALBUM,URI,ANALYZED,DANCEABILITY,ENERGY,KEY,LOUDNESS,...,VALENCE,TEMPO,TYPE,ID,URI.1,TRACK_HREF,ANALYSIS_URL,DURATION_MS,TIME_SIGNATURE,PLAYLIST
0,0,Perfect Form - Henry Krinkle Remix,Cyril Hahn/Shy Girls/Henry Krinkle,Perfect Form EP,NaN,NO,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,House Slow
1,1,Se Acaba el Tiempo,Antaares/Elepani,Se Acaba el Tiempo,spotify:track:32eOsmgDHGkwPfwm3KEVTr,MAYBE,0.775,0.405,7.0,-12.400,...,0.0348,190.011,audio_features,32eOsmgDHGkwPfwm3KEVTr,spotify:track:32eOsmgDHGkwPfwm3KEVTr,https://api.spotify.com/v1/tracks/32eOsmgDHGkw...,https://api.spotify.com/v1/audio-analysis/32eO...,447158.0,4.0,House Slow
2,2,Sun Goes Down - Original Mix,Marenberg Kollektiv,Circumstance Compilation,spotify:track:5dvceelAVmodeppXEGxytF,YES,0.691,0.212,6.0,-15.583,...,0.1160,93.020,audio_features,5dvceelAVmodeppXEGxytF,spotify:track:5dvceelAVmodeppXEGxytF,https://api.spotify.com/v1/tracks/5dvceelAVmod...,https://api.spotify.com/v1/audio-analysis/5dvc...,344000.0,4.0,House Slow
3,3,Kiss Me,Le Flex,"I'll Do My Best, But I Can't Promise Anything",spotify:track:7m6D83BezZRNoZ06bBeRsg,YES,0.698,0.358,9.0,-10.928,...,0.5920,83.996,audio_features,7m6D83BezZRNoZ06bBeRsg,spotify:track:7m6D83BezZRNoZ06bBeRsg,https://api.spotify.com/v1/tracks/7m6D83BezZRN...,https://api.spotify.com/v1/audio-analysis/7m6D...,297175.0,4.0,House Slow
4,4,You Make Me Feel Good - Deep Mix,Satin Jackets,You Make Me Feel Good,spotify:track:5nOo4mBYr11vrDBZxFX33T,YES,0.782,0.502,2.0,-7.629,...,0.0635,113.998,audio_features,5nOo4mBYr11vrDBZxFX33T,spotify:track:5nOo4mBYr11vrDBZxFX33T,https://api.spotify.com/v1/tracks/5nOo4mBYr11v...,https://api.spotify.com/v1/audio-analysis/5nOo...,526569.0,4.0,House Slow


In [15]:
df_similarity = df.dropna()
feature_cols=[
            'ACOUSTICNESS',
             'DANCEABILITY',
             'DURATION_MS',
             'ENERGY',
             'INSTRUMENTALNESS',
             'KEY',
             'LIVENESS',
             'LOUDNESS',
             'MODE',
             'SPEECHINESS',
             'TEMPO',
             'TIME_SIGNATURE',
             'VALENCE']

similarity_cols=[
            # 'ACOUSTICNESS',
             'DANCEABILITY',
             # 'DURATION_MS',
             'ENERGY',
             'INSTRUMENTALNESS',
             'KEY',
             'LIVENESS',
             # 'LOUDNESS',
             # 'MODE',
             'SPEECHINESS',
             'TEMPO',
             # 'TIME_SIGNATURE',
             'VALENCE']

scaler = MinMaxScaler()
normalized_df =scaler.fit_transform(df_similarity[feature_cols])
# Create a pandas series with song titles as indices and indices as series values 
indices = pd.Series(df_similarity.dropna().reset_index(drop=True).index, index=df_similarity['TITLE'])#.drop_duplicates()
# Create cosine similarity matrix based on given matrix
cosine = cosine_similarity(normalized_df)
sig_kernel = sigmoid_kernel(normalized_df)

In [16]:
def generate_recommendation(song_title, model_type=cosine):
        """
        Purpose: Function for song recommendations 
        Inputs: song title and type of similarity model
        Output: Pandas series of recommended songs
        """
        # Get song indices
        index=indices[song_title]
        # Get list of songs for given songs
        
        # print(index)
        score=list(enumerate(model_type[indices[song_title]]))
        # Sort the most similar songs
        similarity_score = sorted(score,key = lambda x:x[1],reverse = True)
        # Select the top-10 recommend songs
        similarity_score = similarity_score[1:11]
        top_songs_index = [i[0] for i in similarity_score]
        # Top 10 recommende songs
        top_songs=df_similarity['TITLE'].iloc[top_songs_index]
        return top_songs
    


In [17]:
def next_track(current_track = 'Misirlou'):
    recommendations_cos = np.array(generate_recommendation(current_track,cosine).values)
    recommendations_sig = np.array(generate_recommendation(current_track,sig_kernel).values)
    df_similarity = pd.DataFrame(data = {'Cosine Similarity':recommendations_cos,
                                      'Sigmoid Kernel':recommendations_sig})#,columns = ['Cosine Similarity','Sigmoid Kernel'],)
    return df_similarity

In [20]:
current_track='For Tahn'
next_track(current_track)

,Cosine Similarity,Sigmoid Kernel
0,Kindness in Return,You Were - Original Mix
1,Above The Clouds,High - Original Mix
2,Girnata Swing,Dragon Walker
3,Girnata Swing,Girnata Swing
4,Mihrap - Laroz Camel Rider Remix,Girnata Swing
5,Kusskompatibel - Original Version,For Tahn
6,Ederlezi (feat. Dikanda),Con Son - Vocal Mix
7,Toi Et Moi,Above The Clouds
8,Cloudy,Bonjour ibiza - Deep house mix
9,What They Say - Ian Tosel Remix,Alcyon
